# Extract Text from pdfs

In [ ]:
# ENV Vars and imports
import pdfplumber
from pathlib import Path
import pandas as pd
from IPython.display import display

DATA_DIR="./data/sec-10-q/data/v1/docs"



In [ ]:
pdf_paths=[str(file) for file in Path(DATA_DIR).rglob('*') if file.is_file()]

['data/sec-10-q/data/v1/docs/2023 Q1 AMZN.pdf', 'data/sec-10-q/data/v1/docs/2023 Q1 AAPL.pdf', 'data/sec-10-q/data/v1/docs/2022 Q3 AAPL.pdf', 'data/sec-10-q/data/v1/docs/2023 Q2 AMZN.pdf', 'data/sec-10-q/data/v1/docs/2023 Q2 AAPL.pdf', 'data/sec-10-q/data/v1/docs/2023 Q3 MSFT.pdf', 'data/sec-10-q/data/v1/docs/2023 Q2 INTC.pdf', 'data/sec-10-q/data/v1/docs/2023 Q2 MSFT.pdf', 'data/sec-10-q/data/v1/docs/2023 Q2 NVDA.pdf', 'data/sec-10-q/data/v1/docs/2023 Q3 AMZN.pdf', 'data/sec-10-q/data/v1/docs/2023 Q1 NVDA.pdf', 'data/sec-10-q/data/v1/docs/2022 Q3 NVDA.pdf', 'data/sec-10-q/data/v1/docs/2023 Q1 INTC.pdf', 'data/sec-10-q/data/v1/docs/2022 Q3 MSFT.pdf', 'data/sec-10-q/data/v1/docs/2023 Q3 NVDA.pdf', 'data/sec-10-q/data/v1/docs/2023 Q3 AAPL.pdf', 'data/sec-10-q/data/v1/docs/2023 Q1 MSFT.pdf', 'data/sec-10-q/data/v1/docs/2022 Q3 INTC.pdf', 'data/sec-10-q/data/v1/docs/2022 Q3 AMZN.pdf', 'data/sec-10-q/data/v1/docs/2023 Q3 INTC.pdf']


In [18]:
# Things to consider
# How are we handling tables that span multiple pages
# How are we handling multiple columns?


with pdfplumber.open(pdf_paths[0]) as pdf:
    print(f"{pdf_paths[0]} pages: {len(pdf.pages)}")
    text = pdf.pages[0].extract_text()
    print(text)
    example_tables=pdf.pages[3].extract_tables()
    example_df=pd.DataFrame(example_tables[0])
    display(example_df)

   


data/sec-10-q/data/v1/docs/2023 Q1 AMZN.pdf pages: 52
Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
____________________________________
FORM 10-Q
____________________________________
(Mark One)
☒ QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF
1934
For the quarterly period ended March 31, 2023
or
☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF
1934
For the transition period from to .
Commission File No. 000-22513
____________________________________
AMAZON.COM, INC.
(Exact name of registrant as specified in its charter)
____________________________________
Delaware 91-1646860
(State or other jurisdiction of (I.R.S. Employer
incorporation or organization) Identification No.)
410 Terry Avenue North, Seattle, Washington 98109-5210
(206) 266-1000
(Address and telephone number, including area code, of registrant’s principal executive offices)
Securities registered pursuant t

,0,1,2,3,4,5,6
0,Net product sales,,$,"56,455",,$,"56,981"
1,Net service sales,,"59,989",None,,"70,377",None
2,Total net sales,,"116,444",None,,"127,358",None
3,Operating expenses:,,,None,,,None
4,Cost of sales,,"66,499",None,,"67,791",None
5,Fulfillment,,"20,271",None,,"20,905",None
6,Technology and content,,"14,842",None,,"20,450",None
7,Sales and marketing,,"8,320",None,,"10,172",None
8,General and administrative,,"2,594",None,,"3,043",None
9,"Other operating expense (income), net",,249,None,,223,None
